# Tomography reconstruction for PUP_FF_SOFC_hires3_S2

## Objective
Sub-micron resolution with high-energy X-ray tomography

## Sample
* SOFC sample (Cathode, Electrolyte, and Anode)
* First complete/valid scan out of the total seven

## Experiment apparatus
* $\omega \in [-180^\text{o}, 180^\text{o}]$ with $\dot{\omega} = 0.1^\text{o}$
* image resolutoin: $0.2 \mu m / \text{pix}$

# Overview

* Using SVD based method to remove noises in the projection images (after background normalization)
* Using image processing method adaptive histogram equalization ([CLAHE](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization)) to enhance the details in reconstruction results
* Iterative method to compensate the horizontal sample jittering (locally) and drifting (globally)
    * standard phase correlation based rotation center locator
    * iterative global adjustment (IGA)
    * iterative pairwise adjustment (IPA)
* Various atttempt to compensate for the vertical sample jittering (locally) and drifting (globally)

In [5]:
import numpy as np
from IPython.display import IFrame

# SVD-based noise reduction

Singular value decompositionv (SVD) is a powerful tool that is commonly used for:
* noise reduction for realtime image stream
* lossy images compression 
* feature detection. 

In this section, the application of SVD based image enhancement for __tomography reconsutrction__ is investigated.

In [4]:
def svd_enhance(img, eigen_cut=20):
    U, S, V = np.linalg.svd(img, full_matrices=True)
    eigen_cut = min(eigen_cut, U.shape[1], V.shape[0]) 
    return np.dot(U[:,:eigen_cut]*S[:eigen_cut], V[:eigen_cut,:])

> Simple example demonstrating eigen feature (eigenimg)

![eigDemo](imgs/eignimg_2628.gif "eig_400")

## SVD enhanced projections with various n_eig

Left= original image , middle= eigen space, right= reconstructed from reduced eigen space (SVD method)

| `n_eig = 400`   | `n_eig = 300`    |
| -----------   | -------------  |
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | 

| `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------:|
|![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

## SVD enhanced projections with various n_eig

Left= original image , middle= eigen space, right= reconstructed from reduced eigen space (SVD method)

| `n_eig = 40`  | `n_eig = 20` |
| :-----:|:-----------: |
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | 

| `n_eig = 10`    | `n_eig = 5`  |
| :-------------:| :-----:|
|![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |

## SVD enhanced projections with various n_eig

* The first __20__ eigen-vectors are sufficient in capturing the main features in each image.
* $F(I_{i,j}) = I_{i,j}^2$ is use to further suppress the noisy background. 

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | ![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | ![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |


## Tomography reconstruction using SVD enhanced projections

* Following the standard procedure, the rotation center of this data set can be found through phase correlation of the 180 degree pairs. 
* The significant horizontal jittering and drifing of the sample makes it difficult of acquire clear reconstruction results. Therefore, iterative global adjustment (IGA), a horizontal drift adjustment method, is used to __horizontally centering the sample__ from all __3601__ ($-180^\text{o} \to 180^\text{o}$, $\delta\omega=0.1^\text{o}$) images.

> The detailed analysis of IGA will be covered in the next section

With the enhanced and centered projection images, the tomograhy reconstruction can be eaisly done using existing toolkit (tomopy) with the command below

```python
recon = tomopy.recon(projs, thetas, 
                     center=rot_center, 
                     algorithm=recon_config['algorithm'],
                     filter_name=recon_config['filter'],
                    )
```
where the configuration of the reconsutrction is 
```python
recon_config = {'algorithm': 'gridrec',
                'filter'   : 'hann',
               }
```

### Reconstruction results

| `n_eig = 400`   | `n_eig = 300`    | 
| :-----------: |:-------------: | 
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | 

| `n_eig = 160`   | `n_eig = 80`   |
| :-----------:   |:-------------: |
| ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

### Reconstruction results

| `n_eig = 40`  | `n_eig = 20`    | 
| :-----:|:-----------: |
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | 

| `n_eig = 10`    | `n_eig = 5`  |
| :-------------:| :-----:|
| ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

### Reconstruction results

* The __more eigen features (vectors)__ used in the SVD enhancement, __the sharper__ the images. 
    * this effect is not linearly (__non-linearity__) depending no n_eig as the increase in sharpness plateaued quickly when n_eig passed __20__.

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

### Reconstruction results

* The ring artifact (ripple features) are more prominent with larger n_eig.  
    * when $n_\text{eig}=5$, the ring artifacts almost disappearted.
    * the preojction images reconstructed from the __first 5 eigen vectors__ looks very different from the original images (see previous sections), but the reconstruction results are __roughly the same__ as the other.

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

### Reconstruction results

* A significant portion (~10%) of the pixels in the reconstruction images have negative values (natural outcome of tomopy)
* The CDF distribution remains very steady, regardless of the number of eigen features used for reconstruction.
    
| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

## Summary (SVD-based noise reduction)

Overall, the SVD based denoising method proves to be efficient in removing the unwanted background noises in the normalized images.

* Pros
    * efficient in removing unwanted noises in the normilzed images
    * easy to implement
    * only need first 20 eigen vectors
* Cons
    * fine details (pixel level) might lost in the process
    * high computation cost
    * might introduce artifacts in the final results

> NOTE:
The SVD based enhancement also has very little effect for the sample jittering/drifting adjustment, the details of which are covered in the next section.

# Enhance reconstruction results with adaptive histogram equilization

The tomography reconstruction results from high energy xray diffraction often has low contrast, making it difficult to spot the important features through visual inspection.
To overcome this issue, adaptive histogram equilization ([CLAHE](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization)) is used to bring out the details in the reconstruction results.
An __interactive__ example is provided in the next cell.

In [16]:
IFrame(src='docs/demoCLAHE.html', width=550, height=500)

__NOTE__  
several features about CLAHE need to be pointed out here:

* CLAHE does not add _new_ features, it only amplify features based on local histogram
* CLAHE does not remove artifacts.
    * It might actually amplify the artifacts
* CLAHE can be computational expensive
    * The CLAHE performed here is using the CLAHE function provided by ImageJ

## Horizontal sample jittering and drifting

As mentioned in previous section, significant amount of sample jittering (locally) and drifting (globally) were found along the horizontal axis. 
In other words, the rotation center for each image are not necessearily correlated anymore.
Therefore, it is necesary to properly align samples from different images ($\omega$) such that a universal rotation center can be defined for the reconstruction process. 

To this end, two different method are proposed here:

* iterative global adjustment (IGA)
    * calcuate the rotation center ($y_{rc}^\omega$) of each pair image ($\Delta\omega = 180^\text{o}$)
    * find the average rotatio center $\bar{y}_{rc}$ 
    * move the rotation center of each pair to the center column using $\bar{y}_{rc}$
    * calculate new average rotatio center $y_{rc}'$
    * repeat until $y_{rc}$ converges
* iterative pairwise adjustment (IPA)
    * calcuate the rotation center ($y_{rc}^\omega$) of each pair image ($\Delta\omega = 180^\text{o}$)
    * move rotation center of each image to the center column using $y_{rc}^\omega$
    * calculate average rotatio center $y_{rc}'$
    * repeat until $y_{rc}$ converges

### Profiling the horizontal jittering

Using phase correlation on image pairs that are $180^\text{o}$ away along $\omega$-axis, individual rotation centers $y_{rc}^\omega$ can be located for all images, the distribution of which can be used to evaluate the horizontal misalignment for the projections.

In [17]:
IFrame(src='imgs/rotcnt_stats.pdf', width=850, height=500)

The distribution shows 
* large variance of $y_c$ in raw data (~20 pixels).

In [18]:
IFrame(src='imgs/rotcnt_stats.pdf', width=850, height=500)

The distribution shows 
* IGA brings the average rotation center ($\bar{y}_{rc}^\omega$) down to the image column center (250).
    * this has little effect of the variance of $y_{rc}^\omega$.

In [19]:
IFrame(src='imgs/rotcnt_stats.pdf', width=850, height=500)

The distribution above shows
* IPA brings almost all the rotation centers to 250, except for six $\omega$s (3 pairs).
    * Further investigation reveals that these three _uncorrectable_ pairs contains at least one corrupted image, which need to be exculded.
    * IPA also serves as an image corruption detector as its response to corrupted images is drastically different from the proper images.

In [20]:
IFrame(src='imgs/rotcnt_stats.pdf', width=750, height=500)

### Original v.s. IPA

* Reconstruction results for original case and IGA case are both enhanced with SVD (first 20 eigen vectors) and CLAHE  
* The side view is the central cross slice generated using ImageJ

In [22]:
print("Original v.s. IGA (top view, XZ in aps)")
IFrame(src='docs/demoIGA.html', width=550, height=500)

Original v.s. IGA (top view, XZ in aps)


In [23]:
print("Original v.s. IGA (side view, XY in aps)")
IFrame(src='docs/demoIGA_YZ.html', width=550, height=500)

Original v.s. IGA (side view, XY in aps)


In [25]:
print("Original v.s. IGA (side view, YZ in aps)")
IFrame(src='docs/demoIGA_XZ.html', width=550, height=400)

Original v.s. IGA (side view, YZ in aps)


### Original v.s. IPA

* Similar to previous section, the reconstruction results from the uncorrected data (Original) are compared with those corrected (horizontally aligned) using IPA.  The side views indicate that the proposed horizontal alignment can significantly imporove the reconstructed geometry as well as reducing the ambient noise from the reconstruction process.  However, geometrical distortion of the sample is still presetn even after the IPA based horizontal correction, suggesting that there are other factors that need to be addressed in order to further improve the reconstruction quality.

> Similar to the previous section, the side view is taken at the center (central cross-slice).